In [7]:
import pandas as pd
import requests
import sqlalchemy
import keys

In [8]:
host = '127.0.0.1'
schema = 'gans'
user = 'root'
password = 'put_your_own_password'
port = 3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [9]:
city_df = pd.read_sql('city',con=con)
city_df

,city_id,city,country,population,lat,long
0,1,Istanbul,Turkey,15840900,41.0136,28.95500
1,2,Moscow,Russia,12632409,55.7500,37.61670
2,3,London,United Kingdom,8799800,51.5072,-0.12750
3,4,Saint Petersburg,Russia,5376672,59.9500,30.30000
4,5,Berlin,Germany,3677472,52.5167,13.38330
5,6,Madrid,Spain,3305408,40.3833,-3.71667
6,7,Kyiv,Ukraine,2962180,50.4500,30.52330
7,8,Rome,Italy,2761632,41.9000,12.50000
8,9,Baku,Azerbaijan,2303100,40.4000,49.90000
9,10,Bucharest,Romania,2161347,44.4325,26.10390


In [15]:
API_key = 'put_your_own_key'

In [24]:
weather_dict = {'city_id':[],
                'time':[],
                'temp':[],
                'wind_speed':[]}

In [25]:
for i, city in enumerate(city_df['city_id']):
    url = f"http://api.openweathermap.org/data/2.5/forecast?lat={city_df.iloc[i]['lat']}&lon={city_df.iloc[i]['long']}&appid={API_key}&units=metric"
    response = requests.get(url)
    weather_json = response.json()
    for time in weather_json['list'][:9]:
        weather_dict['city_id'].append(city)
        weather_dict['time'].append(time['dt_txt'])
        weather_dict['temp'].append(time['main']['temp'])
        weather_dict['wind_speed'].append(time['wind']['speed'])

In [26]:
weather_df = pd.DataFrame(weather_dict)
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 324 entries, 0 to 323
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   city_id     324 non-null    int64  
 1   time        324 non-null    object 
 2   temp        324 non-null    float64
 3   wind_speed  324 non-null    float64
dtypes: float64(2), int64(1), object(1)
memory usage: 10.2+ KB


In [27]:
weather_df

,city_id,time,temp,wind_speed
0,1,2023-01-06 12:00:00,11.52,3.38
1,1,2023-01-06 15:00:00,11.45,3.06
2,1,2023-01-06 18:00:00,10.51,1.42
3,1,2023-01-06 21:00:00,9.80,0.91
4,1,2023-01-07 00:00:00,9.14,1.40
...,...,...,...,...
319,36,2023-01-07 00:00:00,-6.74,7.65
320,36,2023-01-07 03:00:00,-8.87,7.68
321,36,2023-01-07 06:00:00,-11.69,7.49
322,36,2023-01-07 09:00:00,-10.94,6.92


In [28]:
weather_df.to_sql('weather',con=con,if_exists='append',index=False)

324